# Geração dos parâmetros para o DH
Os parâmetros do DH são gerados previamente e são do conhecimento dos intervenientes na comunicação. No caso de haver múltiplas sessões ao longo do tempo estes parâmetros podem ser reutilizados.

In [1]:
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives.asymmetric import dh
from cryptography.hazmat.primitives import serialization
from pickle import dumps, loads


# Generate some parameters. These can be reused.
parameters = dh.generate_parameters(generator=2, key_size=3072,
                                     backend=default_backend())

# Geração do par de chaves DSA
O par de chaves utilizados no DSA é gerado previamente e permanece inalterado ao longo do tempo. Neste projeto, como não são utilizados certificados, assume-se que as chaves públicas já foram partilhadas e os intervenientes têm acesso às que necessitam.

As chaves geradas têm um comprimento de 3072 bit e são, posteriormente, guardadas em ficheiro. De realçar que o ficheiro com a chave privada é cifrado utilizando uma cifra que tem como *input* uma *password* inserida pelo utilizador.

In [2]:
from getpass import getpass
from cryptography.hazmat.primitives.asymmetric import dsa

def gen_dsa(pk_file="pk.pem",pub_file="pub.pem"):
    password = bytes(getpass('password '),'utf-8')
    
    # Gerar parâmetros DSA
    dsa_parameters = dsa.generate_parameters(key_size=3072, backend=default_backend())
    
    # Gerar chave privada
    pk = dsa_parameters.generate_private_key()
    pk_pem = pk.private_bytes(
        encoding=serialization.Encoding.PEM,
        format=serialization.PrivateFormat.PKCS8,
        encryption_algorithm=serialization.BestAvailableEncryption(password)
    )
    
    # Gerar chave pública
    pub = pk.public_key()
    pub_pem = pub.public_bytes(
        encoding=serialization.Encoding.PEM,
        format=serialization.PublicFormat.SubjectPublicKeyInfo
    )

    # Escrever em ficheiros as chaves geradas
    with open(pk_file,"wb") as fp:
        fp.write(pk_pem)
    with open(pub_file,"wb") as fp:
        fp.write(pub_pem)
        
gen_dsa("emitter_pk.pem", "emitter_pub.pem")
gen_dsa("receiver_pk.pem", "receiver_pub.pem")

password ········
password ········


# Função responsável por gerar o MAC de um criptograma

Para o efeito foi utilizado o `HMAC_SHA256` e a chave gerada para produzir um MAC que identifica unicamente o criptograma em questão. Por um lado, o emissor utiliza esta função para produzir o HMAC que é enviado juntamente com o criptograma. Por outro lado, o recetor utiliza-a com o objetivo de verificar o HMAC associado ao criptograma recebido e, consequentemente, verificar a integridade da mensagem.

In [3]:
from cryptography.hazmat.primitives import hashes, hmac, cmac
from cryptography.hazmat.backends import default_backend

def generate_mac(key, crypto):
    h = hmac.HMAC(key, hashes.SHA256(), backend = default_backend())
    h.update(crypto)
    return h.finalize()

# Funções auxiliares
- **Hash**: função que realiza o hash de um dado *input* utilizando a função de hash criptográfica `SHA256`.
- **my_mac**: permite gerar um mecanismo que permite autenticar mensagens, com recurso a cifras por blocos que utilizam uma `key`.

In [4]:
default_algorithm = hashes.SHA256
def Hash(s):
    digest = hashes.Hash(default_algorithm(),backend=default_backend())
    digest.update(s)
    return digest.finalize()

def my_mac(key):
    return cmac.CMAC(algorithms.AES(key), backend=default_backend())

# Geração de Tweaks
A geração de tweaks segue a especificação presente no documento [Tweakable Block Ciphers](https://people.eecs.berkeley.edu/~daw/papers/tweak-crypto02.pdf).

Assim sendo, os tweaks têm um comprimento de 32 byte e são constituídos da seguinte forma:
- Z0 = N || b || 1
- Zi = N || i || 0, com i != 0

O `N` corresponde a um *nonce* gerado aleatoriamente, que não é reutilizado (`gen_nonce`).

O `i` corresponde ao número do *tweak* a ser gerado.

O `b` corresponde ao comprimento da mensagem em bits.

In [9]:
import os, io
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.primitives.kdf.hkdf import HKDF
from cryptography.exceptions import *

USED_NONCES = []
BLOCK_SIZE = 32

def bytes_xor(bytes1, bytes2):
    return b''.join([(a^b).to_bytes(1,byteorder='big') for (a,b) in zip(bytes1, bytes2)])

def gen_nonce():
    nonce = os.urandom(BLOCK_SIZE // 2)
    while nonce in USED_NONCES:
        nonce = os.urandom(BLOCK_SIZE // 2)
    USED_NONCES.append(nonce)
    return nonce

def modifyBit( n,  p,  b): 
    mask = 1 << p 
    return (n & ~mask) | ((b << p) & mask) 

# i is the tweak number
def gen_tweak(i, plaintext_len=0):
    if i > 0:
        nonce = gen_nonce() # Generates nonce
        # Concate between nonce with i [N||i]
        tweak = nonce + int(i).to_bytes(BLOCK_SIZE // 2, byteorder='big')
        tweak = int.from_bytes(tweak, byteorder='big') # Convert to int
        tweak = tweak >> 1 # Removes last bit
        tweak = tweak << 1 # Adds 0 bit in the last position [N||i||0]
        tweak = tweak.to_bytes(BLOCK_SIZE, byteorder='big') # Conver to bytes
    else:
        nonce = gen_nonce() # Generates nonce
        # Concate between nonce with plaintext_len [N||b]
        tweak = nonce + plaintext_len.to_bytes(BLOCK_SIZE // 2, byteorder='big')
        tweak = int.from_bytes(tweak, byteorder='big') # Convert to int
        # Adds 1 bit in the last position [N||b||1]
        tweak = modifyBit(tweak, 0, 1)
        tweak = tweak.to_bytes(BLOCK_SIZE, byteorder='big') # Convert to bytes    
    return tweak

# Emissor

O processo `Emitter` anima os seguintes passos:

1. Carrega o conteúdo dos ficheiros que têm as chaves do DSA. De realçar que a variável *password* tem o seu conteúdo *hardcoded* uma vez que, se assim não fosse, os processos ficavam em *deadlock* à espera para utilizar o *input* para receber os dados do utilizador. Na prática utilizaria-se a funçao `getpass` para que o utilizador pudesse introduzir a *password* para aceder à sua chave privada.
2. Gera o par de chaves do DH (chave privada: x; chave pública: g^x), através dos parâmetros gerados com antecedência.
3. Protocolo DH:
    1. **Emissor -> Receptor**: g^x
    2. **Emissor <- Receptor**: { { g^y, E_k(S(g^y, g^x)), nonce, tag }, mac }
    3. Gera o segredo partilhado: g^xy
    4. Confirmação da integridade da mensagem, através da validação do mac.
    5. Confirmação que o segredo partilhado (g^xy) é o mesmo, através da validação da tag.
    6. Gera a **KEY** através do segredo partilhado.
    7. Decifra a assinatura S.
    8. Verifica que a assinatura é a mesma para (g^y || g^x).
    9. Gera a assinatura para (g^x || g^y).
    10. Cifra a assinatura anteriormente gerada.
    11. Gera o mac da mensagem.
    12. **Emissor -> Receptor**: { { E_k(S(g^x, g^y)), nonce, tag }, mac }
4. Comunicação cifrada:
    1. Recebe um bloco da mensagem a ser enviada.
    2. Atualiza o valor do *checksum*: *checksum* XOR bloco.
    3. Gera o *tweak* correspondente ao bloco e atualiza o número do *tweak*.
    4. **Emissor -> Receptor**: tweak
    5. Cifra o bloco.
    6. Atualiza o MAC.
    7. **Emissor -> Receptor**: (ciphertext, mac)
5. Último bloco:
    1. Gera o *tweak* do bloco de paridade.
    2. Realiza o XOR entre o *checksum* e o *tweak* gerado.
    2. Cifra o bloco que origina a tag.
    3. **Emissor -> Receptor**: tweak
    4. **Emissor -> Receptor**: (cipher.finalize(), mac.finalize(), tag)
6. Termina a execução.

In [10]:
from cryptography.hazmat.primitives.serialization import Encoding
from cryptography.hazmat.primitives.serialization import PublicFormat

message_size = 2**10

def Emitter(conn):
    
    # Ler chaves DSA
    password = b'1234'
    with open("emitter_pk.pem", "rb") as fp:
        dsa_pk = serialization.load_pem_private_key(
            fp.read(),
            password=password,
            backend=default_backend())
        
    with open("receiver_pub.pem", "rb") as fp:
        peer_dsa_pub = serialization.load_pem_public_key(
            fp.read(),
            backend=default_backend())
    
    # Geração das chaves do DH
    pk = parameters.generate_private_key()
    pub_der = pk.public_key().public_bytes(Encoding.DER, PublicFormat.SubjectPublicKeyInfo)
    
    # Emitter -> Receiver : g^x
    conn.send(pub_der)
    
    # Emissor <- Receptor : (g^y, E_k(S(g^y, g^x)), nonce, tag) e mac
    info_receiver = conn.recv()
    peer_package = info_receiver['package']
    mac = info_receiver['mac']
    
    peer_pub = serialization.load_der_public_key(
        peer_package['pub_der'],
        backend=default_backend())
    
    # Geração do segredo partilhado : g^xy
    shared_key = pk.exchange(peer_pub)
    
    # Confirmação do MAC
    try:
        new_mac = generate_mac(shared_key, dumps(peer_package))
        if new_mac != mac:
            raise Exception("MAC não é igual.")
    except Exception as err:
        print("Erro no emissor: {0}".format(err))
        exit(1)
    
    # Confirmação que o segredo partilhado é o mesmo
    try:
        my_tag = Hash(bytes(shared_key))
        if my_tag != peer_package['tag']:
            raise Exception("DH tag não são iguais.")
    except Exception as err:
        print("Erro no emissor: {0}".format(err))
        exit(1)
    
    # Geração de uma chave através do segredo partilhado
    key = HKDF(
        algorithm=hashes.SHA256(),
        length=BLOCK_SIZE*2,
        salt=None,
        info=b'handshake data',
        backend=default_backend()
    ).derive(shared_key)
    
    # nonce para inicialização do modo CRT: devem ser distintos e aleatórios
    nonce  = peer_package['nonce']
    
    # Decifrar a assinatura do receptor
    cipher = Cipher(algorithms.AES(key[:BLOCK_SIZE]), modes.CTR(nonce), 
                    backend=default_backend()).decryptor()
    peer_sig = cipher.update(peer_package['crypto']) + cipher.finalize()
    
    # Verificação da assinatura DSA do receptor
    try:
        peer_dsa_pub.verify(peer_sig, peer_package['pub_der'] + pub_der, hashes.SHA256()) # g^y + g^x
    except InvalidSignature as err:
        print('Erro no emissor: {0}'.format(err))
        exit(1)
        
    # Geração da assinatura DSA do emissor : S(g^x, g^y)
    sig = dsa_pk.sign(pub_der + peer_package['pub_der'], hashes.SHA256())
        
    # nonce para inicialização do modo CRT: devem ser distintos e aleatórios
    nonce  = os.urandom(BLOCK_SIZE // 2)
    
    # Cifrar a assinatura do emissor
    cipher = Cipher(algorithms.AES(key[:BLOCK_SIZE]), modes.CTR(nonce), 
                    backend=default_backend()).encryptor()
    crypto = cipher.update(sig) + cipher.finalize()
    
    # Envio do (E_k(S_A(g^x, g^y)), nonce e tag) e mac
    package = { 'crypto': crypto, 'nonce': nonce, 'tag': my_tag }
    info = { 'package': package, 'mac': generate_mac(shared_key, dumps(package)) }
    conn.send(info)
    
    # Cria um input stream com um número grande de bytes aleatórios
    inputs = io.BytesIO(bytes('1'*message_size,'utf-8'))

    # nonce para inicialização do modo CRT: devem ser distintos e aleatórios
    nonce  = os.urandom(BLOCK_SIZE // 2)
    
    # Dados associados
    dados = bytes('Dados associados desta comunicação','utf-8')
    
    # Geração do contexto de cifra
    cipher = Cipher(algorithms.AES(key[:BLOCK_SIZE]), modes.CTR(nonce), 
                    backend=default_backend()).encryptor()

    # Escolher um mac e inicializa-lo com os dados associados
    mac = my_mac(key[BLOCK_SIZE:])
    mac.update(dados)

    # comunicação e operação de cifra    
    conn.send(nonce) # envia pela conexão o nonce
    # define um buffer para onde vão ser lidos, sucessivamente, os vários blocos do input
    buffer = bytearray(BLOCK_SIZE) 
    
    tweak_num = 1
    checksum = b''
    
    # lê, cifra e envia sucessivos blocos do input 
    try:     
        while inputs.readinto(buffer):
            if tweak_num == 1:
                checksum = bytes(buffer)
            else:
                checksum = bytes_xor(checksum, bytes(buffer))
            tweak = gen_tweak(tweak_num) # gera o tweak para o buffer correspondente
            tweak_num += 1
            conn.send(tweak) # envia o tweak
            ciphertext = cipher.update(bytes_xor(cipher.update(bytes(buffer)), tweak))
            mac.update(ciphertext)
            conn.send((ciphertext, mac.copy().finalize(), None))   
        # Geração do bloco de paridade
        tweak = gen_tweak(0, BLOCK_SIZE)
        tag = cipher.update(bytes_xor(cipher.update(checksum), tweak))
        conn.send(tweak)
        conn.send((cipher.finalize(), mac.finalize(), tag))    # envia a finalização
    except Exception as err:
        print("Erro no emissor: {0}".format(err))
        
    inputs.close()          # fecha a 'input stream'
    conn.close()            # fecha a conecção

# Receptor

O processo `Receiver` anima os seguintes passos:

1. Carrega o conteúdo dos ficheiros que têm as chaves do DSA. De realçar que a variável *password* tem o seu conteúdo *hardcoded* uma vez que, se assim não fosse, os processos ficavam em *deadlock* à espera para utilizar o *input* para receber os dados do utilizador. Na prática utilizaria-se a funçao `getpass` para que o utilizador pudesse introduzir a *password* para aceder à sua chave privada.
2. Gera o par de chaves do DH (chave privada: y; chave pública: g^y), através dos parâmetros gerados com antecedência.
3. Protocolo DH:
    1. **Emissor -> Receptor**: g^x
    2. Gera o segredo partilhado: g^xy
    3. Gera a **KEY** através do segredo partilhado.
    4. Gera a assinatura para (g^y || g^x).
    5. Cifra a assinatura anteriormente gerada.
    6. **Emissor <- Receptor**: { { g^y, E_k(S(g^y, g^x)), nonce, tag }, mac }
    7. **Emissor -> Receptor**: { { E_k(S(g^x, g^y)), nonce, tag }, mac }
    8. Gera o mac da mensagem.
    9. Confirmação da integridade da mensagem, através da validação do mac.
    10. Confirmação que o segredo partilhado (g^xy) é o mesmo, através da validação da tag.
    11. Decifra a assinatura S.
    12. Verifica que a assinatura é a mesma para (g^x || g^y).
4. Comunicação cifrada:
    1. **Emissor -> Receptor**: tweak
    2. **Emissor -> Receptor**: (ciphertext, mac)
    3. Validação do MAC.
    4. Realiza o XOR entre o `ciphertext` e o `tweak`.
    5. Decifra o `ciphertext`.
    6. Atualiza o valor do *checksum*: *checksum* XOR `plaintext`.
5. Último bloco:
    1. Validação do MAC.
    2. Geração da tag através do *checksum*.
    3. Validação do bloco de paridade.
    4. Retorna a mensagem recebida.
6. Termina a execução.

In [11]:
def Receiver(conn):
    
    # Ler chaves DSA
    password = b'1234'
    with open("receiver_pk.pem", "rb") as fp:
        dsa_pk = serialization.load_pem_private_key(
            fp.read(),
            password=password,
            backend=default_backend())
    with open("emitter_pub.pem", "rb") as fp:
        peer_dsa_pub = serialization.load_pem_public_key(
            fp.read(),
            backend=default_backend())
    
     # Geração das chaves do DH
    pk = parameters.generate_private_key()
    pub_der = pk.public_key().public_bytes(Encoding.DER, PublicFormat.SubjectPublicKeyInfo)
    
    # Emitter -> Receiver : g^x
    peer_pub_der = conn.recv()
    peer_pub = serialization.load_der_public_key(
        peer_pub_der,
        backend=default_backend())
    
     # Geração do segredo partilhado : g^xy
    shared_key = pk.exchange(peer_pub)
    my_tag = Hash(bytes(shared_key))
    
    # Geração de uma chave através do segredo partilhado
    key = HKDF(
        algorithm=hashes.SHA256(),
        length=BLOCK_SIZE*2,
        salt=None,
        info=b'handshake data',
        backend=default_backend()
    ).derive(shared_key)
    
    # Geração da assinatura DSA do emissor : S_A(g^y, g^x)
    sig = dsa_pk.sign(pub_der + peer_pub_der, hashes.SHA256())
        
    # nonce para inicialização do modo CRT: devem ser distintos e aleatórios
    nonce  = os.urandom(BLOCK_SIZE // 2)
    
    # Cifrar a assinatura do emissor
    cipher = Cipher(algorithms.AES(key[:BLOCK_SIZE]), modes.CTR(nonce), 
                    backend=default_backend()).encryptor()
    crypto = cipher.update(sig) + cipher.finalize()
    
    # Emissor <- Receptor : (g^y, E_k(S_B(g^y, g^x)), nonce, tag) e mac
    package = { 'pub_der': pub_der, 'crypto': crypto, 'nonce': nonce, 'tag': my_tag }
    info = { 'package': package, 'mac': generate_mac(shared_key, dumps(package)) }
    conn.send(info)
    
    
    # Rceber (E_k(S_A(g^x, g^y)), nonce, tag) e mac
    info = conn.recv()
    peer_package = info['package']
    mac = info['mac']
    
    # Confirmação do MAC
    try:
        new_mac = generate_mac(shared_key, dumps(peer_package))
        if new_mac != mac:
            raise Exception("MAC não é igual.")
    except Exception as err:
        print("Erro no receptor: {0}".format(err))
        exit(1)
    
    # Confirmação que o segredo partilhado é o mesmo
    try:
        if my_tag != peer_package['tag']:
            raise Exception("DH tag não são iguais.")
    except Exception as err:
        print("Erro no receptor TAG: {0}".format(err))
        exit(1)
    
    # nonce para inicialização do modo CRT: devem ser distintos e aleatórios
    nonce  = peer_package['nonce']
    
    # Decifrar a assinatura do receptor
    cipher = Cipher(algorithms.AES(key[:BLOCK_SIZE]), modes.CTR(nonce), 
                    backend=default_backend()).decryptor()
    peer_sig = cipher.update(peer_package['crypto']) + cipher.finalize()

    # Verificação da assinatura DSA do emissor
    try:
        peer_dsa_pub.verify(peer_sig, peer_pub_der + pub_der, hashes.SHA256()) # g^x + g^y
    except InvalidSignature as err:
        print('Erro no receptor DSA: {0}'.format(err))
        exit(1)

    # Inicializa um output stream para receber o texto decifrado
    outputs = io.BytesIO()

    # Recuperar a informação de nonce e salt
    nonce = conn.recv()
    
    # Dados associados
    dados = bytes('Dados associados desta comunicação','utf-8')
    
    # Gerar o contexto de decifragem      
    cipher = Cipher(algorithms.AES(key[:BLOCK_SIZE]), modes.CTR(nonce), 
                    backend=default_backend()).decryptor() 

    # escolher um mac e inicializa-lo com os dados associados
    mac = my_mac(key[BLOCK_SIZE:])
    mac.update(dados)
    
    first = True
    
    # operar a cifra: ler da conecção um bloco, autenticá-lo, decifrá-lo e 
    # escrever o resultado no 'stream' de output
    try:
        while True:    
            try:
                tweak = conn.recv()
                buffer, tag, final_tag = conn.recv()
                ciphertext = bytes(buffer)
                mac.update(ciphertext)
                if tag != mac.copy().finalize():
                    raise InvalidSignature("erro no bloco intermédio")
                plaintext = cipher.update(bytes_xor(cipher.update(ciphertext), tweak))
                outputs.write(plaintext)
                if first:
                    checksum = plaintext
                    first = False
                elif plaintext:
                    checksum = bytes_xor(checksum, plaintext)
            except EOFError:
                if tag != mac.finalize():
                    raise InvalidSignature("erro na finalização")
                # Verificação da tag do TAE
                new_tag = cipher.update(bytes_xor(cipher.update(checksum), tweak))
                if new_tag != final_tag:
                    raise Exception("Checksum não é válido.")
                outputs.write(cipher.finalize())
                break
            except InvalidSignature as err:
                raise Exception("autenticação do ciphertext ou metadados: {}".format(err))
        
        print(outputs.getvalue())     # verificar o resultado
        
    except Exception as err:
        print("Erro no receptor: {0}".format(err))
              
    outputs.close()    # fechar 'stream' de output
    conn.close()       # fechar a conecção

# Processos que permitem a comunicação privada síncrona entre um agente Emitter e um agente Receiver

A classe `BiConn` tem como objetivo criar o `pipe` que vai ser usado pelos intervenientes para comunicação, e inicializar os processos que vão utilizar a via de diálogo criada.

In [12]:
from multiprocessing import Process, Pipe

class BiConn(object):
    def __init__(self,left,right,timeout=None):
        """
        left : a função que vei ligar ao lado esquerdo do Pipe
        right: a função que vai ligar ao outro lado
        timeout: (opcional) numero de segundos que aguarda pela terminação do processo
        """
        left_end, right_end = Pipe()
        self.le = left_end
        self.re = right_end
        self.timeout=timeout
        self.lproc = Process(target=left, args=(left_end,))       # os processos ligados ao Pipe
        self.rproc = Process(target=right, args=(right_end,))
        self.left  = lambda : left(left_end)                       # as funções ligadas já ao Pipe
        self.right = lambda : right(right_end)

    def auto(self, proc=None):
        if proc == None:             # corre os dois processos independentes
            self.lproc.start()
            self.le.close()
            self.rproc.start()
            self.lproc.join(self.timeout)
            self.rproc.join(self.timeout)
        else:                        # corre só o processo passado como parâmetro
            proc.start(); proc.join()

    def manual(self):   #  corre as duas funções no contexto de um mesmo processo Python
        self.left()
        self.right()

BiConn(Emitter,Receiver,timeout=30).auto()

b'11111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111